# ex_network_A


Design a 1D neural network of 1000 cells receiving 100 thalamic random inputs at 50, 55, 60, 65, 70 ms.

The thalamo-cx connectivity is one-to-one.

Model of cells is free of choice.

Record and plot spikes, voltage and both the conductances.

In [1]:
# fileName

fileName = 'ex_network_A'

In [2]:
# libs

import pyNN.spiNNaker as sim
from pyNN import space 
import numpy as np
import matplotlib.pyplot as plt


Detected PyNN version 0.9.4 and Neo version 0.6.1


In [3]:
# simulation settings

sim.setup(
        timestep=1, # [ms]
        min_delay=1, # [ms]
        max_delay=100) # [ms]

simtime = 250 # [ms+]


2021-04-22 13:58:23 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2021-04-22 13:58:23 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2021-04-22 13:58:23 INFO: Setting time scale factor to 1.
2021-04-22 13:58:23 INFO: Setting machine time step to 1000 micro-seconds.


['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


In [4]:
# make the network

structure = {}
structure['1D'] = space.Line(
                              dx=1.0, 
                              x0=0.0, 
                              y=0.0, 
                              z=0.0)

sizePop = 100
cx = {}
cx['tot'] = sim.Population(
                    sizePop,
                    sim.IF_cond_exp,
                    cellparams=sim.IF_cond_exp.default_parameters,
                    structure = structure['1D']
                   )



In [5]:
# set recording of state variables

cx['tot'].record(['spikes','v','gsyn_exc','gsyn_inh'])   

In [6]:
# make the thalamic input sources


spike_times = [[]] * sizePop
random_sources = np.random.random_integers(sizePop,size=int(sizePop/10))
for i in random_sources:
    spike_times[i] = [50, 55, 60, 65, 70]


    
fig, axes = plt.subplots(1, 1)
fig.tight_layout(pad=5)
axes_list = fig.axes

for idx, value in enumerate(['network']):
    axes_list[idx].eventplot(spike_times)
    axes_list[idx].set_title('rasterplot of ' + str(value) + ' layer')
    axes_list[idx].set_xlabel('[ms]')
    axes_list[idx].set_ylabel('cells')
    axes_list[idx].set_xlim(0, simtime)

/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: This function is deprecated. Please call randint(1, 100 + 1) instead
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
thalamus = sim.Population(
                        sizePop,
                        sim.SpikeSourceArray(spike_times=spike_times),
                        structure=structure['1D']
                            )

In [8]:
# make the projection

projs = sim.Projection(
                        thalamus,
                        cx['tot'] ,
                        sim.OneToOneConnector(),
                        synapse_type=sim.StaticSynapse(weight=0.1, delay=1),
                        receptor_type='excitatory',
                        space = space.Space(axes = 'x'),
                        label='thal->cx-e'
                        )



In [9]:
# run simulation

sim.run(simtime) 

2021-04-22 13:58:26 INFO: Simulating for 250 1.0ms timesteps using a hardware timestep of 1000us
2021-04-22 13:58:26 INFO: Starting execution process
2021-04-22 13:58:30 INFO: Time 0:00:03.204445 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-04-22 13:58:38 INFO: Time 0:00:08.849312 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-04-22 13:58:43 INFO: Time 0:00:04.905136 taken by PartitionAndPlacePartitioner
Created spalloc job 5962167
2021-04-22 13:58:43 INFO: Created spalloc job 5962167
Waiting for board power commands to complete.
2021-04-22 13:58:43 INFO: Waiting for board power commands to complete.
2021-04-22 13:58:48 INFO: Time 0:00:05.050541 taken by SpallocAllocator
2021-04-22 1

250.0

In [10]:
# save the results


outputs = {}

outputs['cx'] = cx['tot'].get_data()
for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
    cx['tot'].write_data(fileName + '_' + str(recording) + '.pkl')

Getting spikes for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting v for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting gsyn_exc for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting gsyn_inh for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting spikes for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting v for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting gsyn_exc for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting gsyn_inh for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting spikes for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting v for Abstrac

In [11]:
# recover results function

def recover_results(outputs):
    results = {}
    for key in outputs.keys(): # to extract the name of the layer, e.g., Exc, Inh, Thalamus, etc  
        
        # to get voltage and conductances
        for analogsignal in outputs[key].segments[0].analogsignals:
            print(analogsignal.name)
            results[key, analogsignal.name] = analogsignal

        # to get spikes
        results[key, 'spikes'] = outputs[key].segments[0].spiketrains
    return results


In [12]:
# recover results

results = recover_results(outputs)
results.keys()

v
gsyn_exc
gsyn_inh


dict_keys([('cx', 'v'), ('cx', 'gsyn_exc'), ('cx', 'gsyn_inh'), ('cx', 'spikes')])

In [13]:
# check the spikes

fig, axes = plt.subplots(1, 1)
fig.tight_layout(pad=5)
axes_list = fig.axes
idx = 0
axes_list[idx].eventplot(results['cx', 'spikes'])
axes_list[idx].set_title('rasterplot of cx layer')
axes_list[idx].set_xlabel('[ms]')
axes_list[idx].set_ylabel('cells')
axes_list[idx].set_xlim(0, simtime)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 250.0)

In [14]:
# check the voltage

fig, axes = plt.subplots(1, 1)#, figsize=(9,5))
fig.tight_layout(pad=4)
axes_list = fig.axes
idx = 0
im = axes_list[idx].imshow(results['cx', 'v'].T)
axes_list[idx].set_title('voltage of cx')
axes_list[idx].set_xlabel('time [ms]')
axes_list[idx].set_ylabel('cells')
fig.colorbar(im, ax=axes_list[idx], fraction=0.010, label='[mV]')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# check the conductances


for layer in ['cx']:
    fig, axes = plt.subplots(1, 1)
    fig.tight_layout(pad=4)
    fig.suptitle(str(layer) + ' layer')
    axes_list = fig.axes
    
    for idx, gsyn in enumerate(['gsyn_exc']):#, 'gsyn_inh']):
        im = axes_list[idx].imshow(results[layer, gsyn].T)
        axes_list[idx].set_title(str(gsyn))
        axes_list[idx].set_xlabel('time [ms]')
        axes_list[idx].set_ylabel('cells')
        fig.colorbar(im, ax=axes_list[idx], fraction=0.010, label='[uS]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# end simulation

sim.end()